In [1]:
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.insert(0,'/home/nigam/librascal_cs/librascal/build/')

In [2]:
from ase.io import read, write
import ase
import json
from tqdm.notebook import tqdm
class tqdm_reusable:
    def __init__(self, *args, **kwargs):
        self._args = args
        self._kwargs = kwargs

    def __iter__(self):
        return tqdm(*self._args, **self._kwargs).__iter__()
    
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import WignerDReal, ClebschGordanReal, spherical_expansion_reshape, lm_slice, real2complex_matrix, compute_lambda_soap

In [3]:
# a library of utilities and wrapper to compute pair features and manipulate a Hamiltonian-like target
from ncnice import * 

# Check Bispectrum

In [7]:
from ncnice.representations import *#compute_rho3i_lambda

In [8]:
spherical_expansion_hypers = {
    "interaction_cutoff": 4,
    "max_radial": 8,
    "max_angular": 6,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
}

spex = SphericalExpansion(**spherical_expansion_hypers)
mycg = ClebschGordanReal(spherical_expansion_hypers["max_angular"])

In [9]:
frames = read('data/ethanol-structures.xyz', ':')
# frames = read('water/water_randomized_1000.xyz', ':')
# for f in frames:
#     f.cell=[100,100,100]
#     f.positions+=50
#     # use same specie for all atoms so we get a single projection matrix
#     f.numbers = f.numbers*0+1    

# spherical_expansion_hypers = get_optimal_radial_basis_hypers(spherical_expansion_hypers, frames, expanded_max_radial=12)


# # ... and we replicate it 
# pm = spherical_expansion_hypers['optimization']['RadialDimReduction']['projection_matrices'][1] 
# spherical_expansion_hypers['optimization']['RadialDimReduction']['projection_matrices'] = { i: pm for i in range(1000) }
spex = SphericalExpansion(**spherical_expansion_hypers)

In [10]:
frames = read('data/ethanol-structures.xyz', ':')
# frames = read('/water_randomized_1000.xyz', ':')
for f in frames:
    f.cell=[100,100,100]
    f.positions+=50
rhoi = compute_rhoi(frames[:1], spex, spherical_expansion_hypers)*1e0

In [11]:
rho2i_l, prho2i_l = compute_all_rho2i_lambda(rhoi, mycg, rho2i_pca=None)

In [12]:
rho3ilambda, prho3 = compute_rho3i_lambda(rho2i_l,rhoi, 0, mycg, prho2i_l)

0 0 0 0
l1, l2, k, l3, prho2, p, expected 0 0 0 0 1 1 1
0 1 1 0
l1, l2, k, l3, prho2, p, expected 0 1 1 1 1 1 1
0 2 2 0
l1, l2, k, l3, prho2, p, expected 0 2 2 2 1 1 1
0 3 3 0
l1, l2, k, l3, prho2, p, expected 0 3 3 3 1 1 1
0 4 4 0
l1, l2, k, l3, prho2, p, expected 0 4 4 4 1 1 1
0 5 5 0
l1, l2, k, l3, prho2, p, expected 0 5 5 5 1 1 1
0 6 6 0
l1, l2, k, l3, prho2, p, expected 0 6 6 6 1 1 1
1 1 1 1
l1, l2, k, l3, prho2, p, expected 1 1 1 1 -1 -1 -1
1 1 2 1
l1, l2, k, l3, prho2, p, expected 1 1 2 2 1 1 1
1 2 2 2
l1, l2, k, l3, prho2, p, expected 1 2 2 2 -1 -1 -1
1 2 3 1
l1, l2, k, l3, prho2, p, expected 1 2 3 3 1 1 1
1 3 3 2
l1, l2, k, l3, prho2, p, expected 1 3 3 3 -1 -1 -1
1 3 4 1
l1, l2, k, l3, prho2, p, expected 1 3 4 4 1 1 1
1 4 4 2
l1, l2, k, l3, prho2, p, expected 1 4 4 4 -1 -1 -1
1 4 5 1
l1, l2, k, l3, prho2, p, expected 1 4 5 5 1 1 1
1 5 5 2
l1, l2, k, l3, prho2, p, expected 1 5 5 5 -1 -1 -1
1 5 6 1
l1, l2, k, l3, prho2, p, expected 1 5 6 6 1 1 1
1 6 6 2
l1, l2, k, l3, prho2, p, 

In [13]:
#MANUAL computation of bispectrum from librascal/nice_demo.ipynb

selframe = frames[:1];         # frame used for the test
feat_scaling = 1e0         
feats = spex.transform(selframe).get_features(spex)
ref_feats = feat_scaling*spherical_expansion_reshape(feats, **spherical_expansion_hypers)

nice2_full = mycg.combine_nice(ref_feats, ref_feats)
bisp_nice = np.zeros(ref_feats.shape[:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) )
for l in range(spherical_expansion_hypers["max_angular"]+1):
    # while we are at it, we also reorder the indices in a bispectrum-like way
    bisp_nice[...,l] = mycg.combine_einsum(nice2_full[...,lm_slice(l)], 
                                         ref_feats[...,lm_slice(l)], 
                                         L=0, 
                                         combination_string="ianANlL,ibp->ianlANLbp" )[...,0]

In [14]:
bisp_nice[0,0,1,:,0,2,:,0,2,0]

array([[ 2.07141323e-07,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -9.63960119e-08,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.01818857e-07,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.42010029e-07,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  5.18225025e-08,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.82675677e-08,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.1941364

In [15]:
rho3ilambda[0,0,1,0,2,0,2,:,0]

array([ 2.07141323e-07, -9.63960119e-08,  1.01818857e-07, -1.42010029e-07,
        5.18225025e-08, -1.82675677e-08,  1.19413640e-08,  8.76031266e-25,
       -9.11113234e-10, -4.24433872e-09, -6.58195522e-08, -4.43080257e-09,
        4.54856670e-08, -4.04601961e-10, -3.35661883e-09,  1.60444254e-09,
        5.80968589e-09,  8.38271362e-10, -1.84908681e-08, -1.25559488e-24,
        1.86479439e-08,  8.29472454e-08, -3.67235530e-09, -2.30273780e-08,
       -1.57694580e-08, -2.16674422e-09,  9.06422688e-09,  2.39273987e-09,
        6.94772614e-11, -6.75178533e-09, -9.29187773e-26, -5.29462405e-08,
       -2.88929594e-25,  2.36051061e-08, -7.20717436e-10,  1.38368165e-08,
       -1.25046418e-09, -5.66993837e-10, -7.33817985e-09, -1.00277126e-10,
        1.41444235e-08, -1.94206029e-25, -7.20039030e-09, -2.61445379e-09,
       -1.50203059e-11,  4.24586358e-09,  4.46966200e-26,  1.74992743e-09,
        2.61066651e-10, -3.33186302e-09])

they are the **same** modulo noise. because we dont account for sorting l's in the manual computation and the storage format is inefficient, the shapes of the matrices are different

# rho2iP

In [9]:
def compute_rho1ijp_species(rho1ijp, frame, cg): 
    """groups neighbors j by species"""
    species = frame.numbers
    nspecies = len(set(species))
    shape = rho1ijp.shape[:2]+(nspecies,) +rho1ijp.shape[2:]
    rho1ijp_spec = np.zeros(shape)
    
    for ispe, spe in enumerate(sorted(set(species))):
        idx_spe = np.where(species==spe)[0]
#         print(spe, idx_spe)
        for j in range(rho1ijp.shape[1]):
            if j in idx_spe:
                rho1ijp_spec[:,j,ispe, ...]= rho1ijp[:,j,...]
    return rho1ijp_spec

In [10]:
hypers_ij = deepcopy(spherical_expansion_hypers)
hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)

frame = frames[0]
fgij = compute_gij(frame, spex_ij, hypers_ij)
rhoi = compute_rhoi(frame, spex, spherical_expansion_hypers)
rho1ijp, prhoijp = compute_rho1ijp_lambda(rhoi, fgij,0, mycg)
rho1ij, prhoij = compute_rho1ij_lambda(rhoi, fgij,0, mycg)

In [11]:
rho1ijp_l, prho1ijp_l = compute_all_rho1ijp_lambda(rhoi, fgij, mycg)

In [17]:
def compute_rho1ip_lambda(rhoi, rho1ijp_l, fgij, frame, L, cg, prho1ijp_l):
    """compute combined rhoi and rhoi-MP"""
    rho1ijp_spec = {}
    prhoijp_spec = {}
    rho1ip_lambda = {}
    lmax = int(np.sqrt(rhoi.shape[-1])) -1
    for l in range(lmax+1):
        rho1ijp_spec[l] = compute_rho1ijp_species(rho1ijp_l[l], frame, cg)
        rho1ip_lambda[l] = np.sum(rho1ijp_spec[l], axis=1)  # MP - atom density 
    
    # do the usual gig of angular coupling 
    nl=0
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            nl += 1*rho1ip_lambda[l2].shape[4]
# multiplication above to account for (l1,l2,l3) combinations. rho1ip_lambda[L].shape[4] accounts for l1,l2
#  combinations of g(rij) and rhoj
    shape = rhoi.shape[:3] + rho1ip_lambda[L].shape[1:4] +(nl, 2*L+1,)
    rho_combined_lam = np.zeros(shape)
    prho_combined_lam = np.ones(nl, dtype = int)*(1-2*(L%2))
    
    il=0
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            rho_combined_lam[...,il:il+rho1ip_lambda[l2].shape[4],:] = cg.combine_einsum(rhoi[...,lm_slice(l1)], rho1ip_lambda[l2],
                                            L, combination_string="ian,ibNMl->ianbNMl")

            prho_combined_lam[il:il+rho1ip_lambda[l2].shape[4]] = (1-2*(l1%2)) * prho1ijp_l[l2]
            il+=rho1ip_lambda[l2].shape[4]

    return rho_combined_lam, prho_combined_lam

In [22]:
rho_combined_lam, prho_combined_lam = compute_rho1ip_lambda(rhoi, rho1ijp_l, fgij, frame, 1, mycg, prho1ijp_l)

In [23]:
rho_combined_lam.shape

(9, 3, 8, 3, 24, 8, 483, 3)

In [51]:
def compute_hamiltonian_representations(frames, orbs, hypers, lmax, nu, cg, scale=1,
                     select_feats = None, half_hete = True, mp_feats = False,
                     rhoi_pca = None, rho2i_pca = None,
                     rhoij_rho2i_pca = None, rhoij_pca = None,
                     verbose = False
                     ):
    """
        Computes the full set of features needed to learn matrix elements up to lmax.
        Options are fluid, but here are some that need an explanation:

        select_feats = dict(type=["diag", "offd_m", "offd_p", "hete"], block = ('el1', ['el2',] L, pi) )
        does the minimal amount of calculation to evaluate the selected block. other terms might be computed as well if they come for free.
    """

    spex = SphericalExpansion(**hypers)
    rhoi = compute_rhoi(frames, spex, hypers)

    # compresses further the spherical expansion features across species
    if rhoi_pca is not None:
        rhoi = apply_rhoi_pca(rhoi, rhoi_pca)

    # makes sure that the spex used for the pair terms uses adaptive species
    hypers_ij = deepcopy(hypers)
    hypers_ij["expansion_by_species_method"] = "structure wise"
    spex_ij = SphericalExpansion(**hypers_ij)

    tnat = 0
    els = list(orbs.keys())
    nel = len(els)
    # prepare storage
    elL = list(itertools.product(els,range(lmax+1),[-1,1]))
    hetL = [ (els[i1], els[i2], L, pi) for i1 in range(nel) for i2 in range((i1+1 if half_hete else 0), nel) for L in range(lmax+1) for pi in [-1,1] ]
    feats = dict(diag = { L: [] for L in elL },
                 offd_p = { L: [] for L in elL },
                 offd_m = { L: [] for L in elL },
                 hete =   { L: [] for L in hetL },)

    if rhoij_rho2i_pca is None and rho2i_pca is not None:
        rhoij_rho2i_pca = rho2i_pca

    #before = tracemalloc.take_snapshot()
    for f in frames:
        fnat = len(f.numbers)
        frhoi = rhoi[tnat:tnat+fnat]*scale
        fgij = compute_gij(f, spex_ij, hypers_ij)*scale

        if (select_feats is None or select_feats["type"]!="diag") and nu == 2:
            rhonui, prhonui = compute_all_rho2i_lambda(frhoi, cg, rhoij_rho2i_pca)
        else:
            rhonui, prhonui = frhoi, None

        for L in range(lmax+1):
            if select_feats is not None and L>0 and select_feats["block"][-2] != L:
                continue

            if nu==0:
                lrhonui, lprhonui = np.ones((fnat, 1, 2*L+1)), np.ones((1))
            elif nu==1:
                lrhonui, lprhonui = compute_rho1i_lambda(frhoi, L, cg)
            else:
                if mp_feats:
                    frho1ijp_l, fprho1ijp_l = compute_all_rho1ijp_lambda(frhoi, fgij, mycg)
                    lrhonui, lprhonui = compute_rho1ip_lambda(frhoi, frho1ijp_l, fgij, frame, L, cg, fprho1ijp_l)
#                     lrhonui, lprhonui = compute_rho1ip_lambda(frhoi, L, cg)
                else:
                    frho2i_l,fprho2i_l = compute_all_rho2i_lambda(frhoi, cg, rho2i_pca=None)
                    lrhonui, lprhonui = compute_rho3i_lambda(frho2i_l, frhoi, L, cg,fprho2i_l )
                    #lrhonui, lprhonui = compute_rho2i_lambda(frhoi, L, cg)
                    #if rho2i_pca is not None:
                    #     lrhonui, lprhonui = apply_rho2i_pca(lrhonui, lprhonui, rho2i_pca)

            if select_feats is None or select_feats["type"]!="diag":
                if nu==0:
                    lrhoij, prhoij = compute_rho0ij_lambda(rhonui, fgij, L, cg, prhonui)
                elif nu==1:
                    if mp_feats:
                        lrhoij, prhoij = compute_rho1ijp_lambda(rhonui, fgij, L, cg, prhonui)
                    else:
                        lrhoij, prhoij = compute_rho1ij_lambda(rhonui, fgij, L, cg, prhonui)
                else:
                    lrhoij, prhoij = compute_rho2ij_lambda(rhonui, fgij, L, cg, prhonui)
                if rhoij_pca is not None:
                    lrhoij, prhoij = apply_rhoij_pca(lrhoij, prhoij, rhoij_pca)

            for i, el in enumerate(els):
                iel = np.where(f.symbols==el)[0]
                if len(iel) == 0:
                    continue
                if select_feats is not None and el != select_feats["block"][0]:
                    continue

                for pi in [-1,1]:
                    wherepi = np.where(lprhonui==pi)[0]
                    if len(wherepi)==0:
                        # add a vector of zeros
                        feats['diag'][(el, L, pi)].append(np.zeros(shape=(len(iel), 1, 2*L+1)))
                        continue
                    feats['diag'][(el, L, pi)].append(lrhonui[...,wherepi,:][iel].reshape((len(iel), -1, 2*L+1) ) )

                if select_feats is not None and select_feats["type"]=="diag":
                    continue

                triu = np.triu_indices(len(iel), 1)
                ij_up = (iel[triu[0]],iel[triu[1]]) # ij indices, i>j
                ij_lw = (ij_up[1], ij_up[0]) # ij indices, i<j
                lrhoij_p = (lrhoij[ij_up] + lrhoij[ij_lw])/np.sqrt(2)
                lrhoij_m = (lrhoij[ij_up] - lrhoij[ij_lw])/np.sqrt(2)
                for pi in [-1,1]:
                    if len(ij_up[0])==0:
                        continue
                    wherepi = np.where(prhoij==pi)[0];
                    if len(wherepi)==0:
                        feats['offd_p'][(el, L, pi)].append( np.zeros((lrhoij_p.shape[0], 1, 2*L+1)) )
                        feats['offd_m'][(el, L, pi)].append( np.zeros((lrhoij_p.shape[0], 1, 2*L+1)) )
                        continue
                    feats['offd_p'][(el, L, pi)].append(lrhoij_p[...,wherepi,:].reshape(lrhoij_p.shape[0], -1, 2*L+1))
                    feats['offd_m'][(el, L, pi)].append(lrhoij_m[...,wherepi,:].reshape(lrhoij_m.shape[0], -1, 2*L+1))

                if select_feats is not None and select_feats["type"]!="hete":
                    continue
                for elb in els[i+1:]:
                    ielb = np.where(f.symbols==elb)[0]
                    if len(ielb) == 0:
                        continue
                    if select_feats is not None and elb != select_feats["block"][1]:
                        continue

                    # combines rho_ij and rho_ji
                    lrhoij_het = lrhoij[iel][:,ielb]
                    lrhoij_het_rev = np.swapaxes(lrhoij[ielb][:,iel],1,0)
                    # make a copy and not a slice, so we keep better track
                    for pi in [-1,1]:
                        wherepi = np.where(prhoij==pi)[0];
                        if len(wherepi)==0:
                            feats['hete'][(el, elb, L, pi)].append(np.zeros((lrhoij_het.shape[0]*lrhoij_het.shape[1],1,2*L+1)))
                            continue
                        lrhoij_het_pi = lrhoij_het[...,wherepi,:]
                        lrhoij_het_rev_pi = lrhoij_het_rev[...,wherepi,:]
                        feats['hete'][(el, elb, L, pi)].append(
                            np.concatenate([
                            lrhoij_het_pi.reshape(
                                (lrhoij_het.shape[0]*lrhoij_het.shape[1],-1,2*L+1) )
        
                            ,
                            lrhoij_het_rev_pi.reshape(
                                (lrhoij_het_rev.shape[0]*lrhoij_het_rev.shape[1],-1,2*L+1) )
                            ], axis=-2)
                        )
                    #del(lrhoij_het)
                #del(lrhoij_p, lrhoij_m)
            #del(lrhoij, lrho2)
        tnat+=fnat

    # cleans up combining frames blocks into single vectors - splitting also odd and even blocks
    for k in feats.keys():
        for b in list(feats[k].keys()):
            if len(feats[k][b]) == 0:
                continue
            block = np.vstack(feats[k][b])
            feats[k].pop(b)
            if len(block) == 0:
                continue

            feats[k][b] = block.reshape((block.shape[0], -1, 1+2*b[-2]))

    return feats


# Regression Test 

In [52]:
nframes = 50
frames = read('data/ethanol-structures.xyz',':')[:nframes]
for f in frames:
    f.cell=[100,100,100]
    f.positions+=50
    # use same specie for all atoms so we get a single projection matrix,
    # which we can apply throughout. a bit less efficient but much more practical
    f.numbers = f.numbers*0+1
spherical_expansion_hypers = get_optimal_radial_basis_hypers(spherical_expansion_hypers, frames, expanded_max_radial=16)
# ... and we replicate it
pm = spherical_expansion_hypers['optimization']['RadialDimReduction']['projection_matrices'][1]
spherical_expansion_hypers['optimization']['RadialDimReduction']['projection_matrices'] = { i: pm for i in range(99) }
spex = SphericalExpansion(**spherical_expansion_hypers)
hypers_ij = deepcopy(spherical_expansion_hypers)
hypers_ij["expansion_by_species_method"] = "structure wise"
spex_gij = SphericalExpansion(**hypers_ij)


orbs = json.load(open('data/ethanol-saph-orbs.json', "r"))
frames = read('data/ethanol-structures.xyz', ':')[:nframes]
ofocks = np.load('data/ethanol-saph-ofock.npy', allow_pickle=True)[:nframes]

# hamiltonian to block coupling
ofock_blocks, slices_idx = matrix_list_to_blocks(ofocks, frames, orbs, mycg)

# training settings
train_fraction = 0.5
itrain = np.arange(len(frames))
np.random.seed(12345)
np.random.shuffle(itrain)
ntrain = int(len(itrain)*train_fraction)
itest = itrain[ntrain:]; itrain=itrain[:ntrain]
train_slices = get_block_idx(itrain, slices_idx)
print(itrain)

FR = FockRegression(orbs, alpha = 1e-6, #alphas=np.geomspace(1e-8, 1e4, 7),
                    fit_intercept="auto")

for f in frames:
    f.cell=[100,100,100]
    f.positions+=50

print("Calling all representation subroutines (no PCA)")
rhoi = compute_rhoi(frames[0], spex, spherical_expansion_hypers)


[13  8 16  3 15 12  0 10  7 11]
Calling all representation subroutines (no PCA)


In [33]:
import itertools

In [54]:
feats_nu1 = compute_hamiltonian_representations(tqdm_reusable(frames, desc="features", leave=False),
                        orbs, spherical_expansion_hypers, 2, nu=1, cg=mycg, scale=1e3, mp_feats = True)

FR.fit(feats_nu1, ofock_blocks, train_slices, progress=tqdm)
pred_blocks = FR.predict(feats_nu1, progress=tqdm)
pred_ofocks = blocks_to_matrix_list(pred_blocks, frames, slices_idx, orbs, mycg)

mse_train = 0
for i in itrain:
    mse_train += np.sum((pred_ofocks[i] - ofocks[i])**2)/len(ofocks[i])/len(itrain)

mse_test = 0
for i in itest:
    mse_test += np.sum((pred_ofocks[i] - ofocks[i])**2)/len(ofocks[i])/len(itest)

print("Model size: ", len(FR.cv_stats_))
print("Train RMSE: ", np.sqrt(mse_train))
print("Test RMSE: ", np.sqrt(mse_test))


features:   0%|          | 0/20 [00:00<?, ?it/s]

features:   0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

diag (2, 0, 2, 0) ('O', 0, 1) 0 2.7753608473980735
diag (2, 0, 2, 1) ('O', 1, 1) 1 0.4462237741637202
diag (2, 1, 2, 1) ('O', 0, 1) 0 1.8107058087600478
diag (2, 1, 2, 1) ('O', 2, 1) 2 0.7383770695421117
diag (4, 0, 4, 0) ('C', 0, 1) 0 1.5047174907273053
diag (4, 0, 4, 1) ('C', 1, 1) 1 0.17318842920340205
diag (4, 1, 4, 1) ('C', 0, 1) 0 0.6245433127456875
diag (4, 1, 4, 1) ('C', 2, 1) 2 0.1997065011012175
diag (5, 0, 5, 0) ('H', 0, 1) 0 2.3634310716472076


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Model size:  26
Train RMSE:  0.012053177503949585
Test RMSE:  0.05175830885752685


In [55]:
blocks_c = ofock_blocks 
tblock = 'diag'
sel_type  = list(blocks_c[tblock].keys())
fblock=[]
for i in range(len(sel_type)):
    kblock = sel_type[i]
    lblock = list(blocks_c['diag'][sel_type[i]].keys())
    for l in lblock:
        fblock.append(block_to_feat_index(tblock, kblock, l, orbs))

fblock = list(set(fblock))
for j in fblock:
    print(j)
    
    feats_nu1['diag'][j] = compute_hamiltonian_representations(tqdm_reusable(frames, desc="features", leave=False),
                        orbs, spherical_expansion_hypers, 2, nu=2, cg=mycg, scale=1e3, mp_feats = False,  select_feats =dict(block=j, type=tblock))[tblock][j]

('O', 2, 1)


features:   0%|          | 0/20 [00:00<?, ?it/s]

features:   0%|          | 0/20 [00:00<?, ?it/s]

('C', 0, 1)


features:   0%|          | 0/20 [00:00<?, ?it/s]

...

features:   0%|          | 0/20 [00:00<?, ?it/s]

features:   0%|          | 0/20 [00:00<?, ?it/s]

('C', 1, 1)


features:   0%|          | 0/20 [00:00<?, ?it/s]

features:   0%|          | 0/20 [00:00<?, ?it/s]

In [56]:
FR.fit(feats_nu1, ofock_blocks, train_slices, progress=tqdm)
pred_blocks = FR.predict(feats_nu1, progress=tqdm)
pred_ofocks = blocks_to_matrix_list(pred_blocks, frames, slices_idx, orbs, mycg)

mse_train = 0
for i in itrain:
    mse_train += np.sum((pred_ofocks[i] - ofocks[i])**2)/len(ofocks[i])/len(itrain)

mse_test = 0
for i in itest:
    mse_test += np.sum((pred_ofocks[i] - ofocks[i])**2)/len(ofocks[i])/len(itest)

print("Model size: ", len(FR.cv_stats_))
print("Train RMSE: ", np.sqrt(mse_train))
print("Test RMSE: ", np.sqrt(mse_test))


  0%|          | 0/7 [00:00<?, ?it/s]

diag (2, 0, 2, 0) ('O', 0, 1) 0 2.7753608473980735
diag (2, 0, 2, 1) ('O', 1, 1) 1 0.4462237741637202
diag (2, 1, 2, 1) ('O', 0, 1) 0 1.8107058087600478
diag (2, 1, 2, 1) ('O', 2, 1) 2 0.7383770695421117
diag (4, 0, 4, 0) ('C', 0, 1) 0 1.5047174907273053
diag (4, 0, 4, 1) ('C', 1, 1) 1 0.17318842920340205
diag (4, 1, 4, 1) ('C', 0, 1) 0 0.6245433127456875
diag (4, 1, 4, 1) ('C', 2, 1) 2 0.1997065011012175
diag (5, 0, 5, 0) ('H', 0, 1) 0 2.3634310716472076


  0%|          | 0/4 [00:00<?, ?it/s]

offd_p (4, 0, 4, 0) ('C', 0, 1) 0 0.9407257978566762
offd_p (4, 0, 4, 1) ('C', 1, 1) 1 0.06293572470407495
offd_p (4, 1, 4, 1) ('C', 0, 1) 0 0.4455990241628668
offd_p (4, 1, 4, 1) ('C', 2, 1) 2 1.4819606168029316
offd_p (5, 0, 5, 0) ('H', 0, 1) 0 0.5849021565828035


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Model size:  26
Train RMSE:  0.005117145029683599
Test RMSE:  0.09496571883986546


In [57]:
blocks_c = ofock_blocks 
tblock = 'diag'
sel_type  = list(blocks_c[tblock].keys())
fblock=[]
for i in range(len(sel_type)):
    kblock = sel_type[i]
    lblock = list(blocks_c['diag'][sel_type[i]].keys())
    for l in lblock:
        fblock.append(block_to_feat_index(tblock, kblock, l, orbs))

fblock = list(set(fblock))
for j in fblock:
    print(j)
    
    feats_nu1['diag'][j] = compute_hamiltonian_representations(tqdm_reusable(frames, desc="features", leave=False),
                        orbs, spherical_expansion_hypers, 2, nu=2, cg=mycg, scale=1e3, mp_feats = True,  select_feats =dict(block=j, type=tblock))[tblock][j]

('O', 2, 1)


features:   0%|          | 0/20 [00:00<?, ?it/s]

features:   0%|          | 0/20 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
FR.fit(feats_nu1, ofock_blocks, train_slices, progress=tqdm)
pred_blocks = FR.predict(feats_nu1, progress=tqdm)
pred_ofocks = blocks_to_matrix_list(pred_blocks, frames, slices_idx, orbs, mycg)

mse_train = 0
for i in itrain:
    mse_train += np.sum((pred_ofocks[i] - ofocks[i])**2)/len(ofocks[i])/len(itrain)

mse_test = 0
for i in itest:
    mse_test += np.sum((pred_ofocks[i] - ofocks[i])**2)/len(ofocks[i])/len(itest)

print("Model size: ", len(FR.cv_stats_))
print("Train RMSE: ", np.sqrt(mse_train))
print("Test RMSE: ", np.sqrt(mse_test))


# Compute bispectrum as sum over rho2ij lambda - NOT RIGHT

In [ ]:
hypers_ij = deepcopy(spherical_expansion_hypers)
# hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)
scale=1e3
for f in frames[:1]:
    fgij = compute_gij(f, spex_ij, hypers_ij)
    rhoi = compute_rhoi(f, spex, spherical_expansion_hypers)
    rho2i_l, prho2i_l = compute_all_rho2i_lambda(rhoi, mycg, rho2i_pca=None)
    rho2ij, prho2ij = compute_rho2ij_lambda(rho2i_l, fgij, 0, mycg, prho2i_l)
    

In [ ]:
shape = (rho2ij.shape[0],
         rhoi.shape[1],rho2ij.shape[2], 
         rho2ij.shape[3], rho2ij.shape[4], 
         rho2ij.shape[5], rho2ij.shape[6],
         rho2ij.shape[7], rho2ij.shape[8]
        )
bisp_gij = np.zeros(shape)
# parity = np.ones(nl, dtype = int)*(1-2*(L%2))

In [ ]:
rho2ij.shape

In [ ]:
for f in frames[:1]:
    fgij = compute_gij(f, spex_ij, hypers_ij)
    rhoi = compute_rhoi(f, spex, spherical_expansion_hypers)
    rho2i_l, prho2i_l = compute_all_rho2i_lambda(rhoi, mycg, rho2i_pca=None)
    rho2ij, prho2ij = compute_rho2ij_lambda(rho2i_l, fgij, 0, mycg, prho2i_l)
    shape = (rho2ij.shape[0],
         rhoi.shape[1],rho2ij.shape[2], 
         rho2ij.shape[3], rho2ij.shape[4], 
         rho2ij.shape[5], rho2ij.shape[6],
         rho2ij.shape[7], rho2ij.shape[8]
        )
    bisp_gij = np.zeros(shape)
#     parity = np.ones(nl, dtype = int)*(1-2*(L%2))
    species = f.numbers
    nspecies = rhoi.shape[1]
    for ispe, spe in enumerate(set(species)):
        idx_spe = np.where(species==spe)[0]
        print(spe, idx_spe)
        bisp_gij[:,ispe, ...]= np.sum(rho2ij[:,idx_spe,...],axis =1)

In [ ]:
bisp_gij.shape

In [ ]:
bisp_gij[0,0,1,0,2,0,2,:,0][np.where(bisp_gij[0,0,1,0,2,0,2,:,0])]

In [ ]:
rho3ilambda[0,0,1,0,2,0,2,:,0][np.where(rho3ilambda[0,0,1,0,2,0,2,:,0])]

In [ ]:
rho3ilambda[0][np.where(rho3ilambda[0])]

In [ ]:
bisp_gij[0][np.where(bisp_gij[0])]*2.14dd

In [ ]:
bisp_gij.shape

In [ ]:
def compute_mp(frames,hypers, lmax, nui, nuj, cg, rhoi_pca=None, rhoij_rho2i_pca=None,rho2i_pca=None, scale=1):
    spex = SphericalExpansion(**hypers)
    rhoi = compute_rhoi(frames, spex, hypers)

    # compresses further the spherical expansion features across species
    if rhoi_pca is not None:
        rhoi = apply_rhoi_pca(rhoi, rhoi_pca)

    # makes sure that the spex used for the pair terms uses adaptive species
    hypers_ij = deepcopy(hypers)
    hypers_ij["expansion_by_species_method"] = "structure wise"
    spex_ij = SphericalExpansion(**hypers_ij)

    tnat = 0
    els = list(orbs.keys())
    nel = len(els)
    # prepare storage
#     elL = list(itertools.product(els,range(lmax+1),[-1,1]))
#     hetL = [ (els[i1], els[i2], L, pi) for i1 in range(nel) for i2 in range((i1+1 if half_hete else 0), nel) for L in range(lmax+1) for pi in [-1,1] ]
#     feats = dict(diag = { L: [] for L in elL },
#                  offd_p = { L: [] for L in elL },
#                  offd_m = { L: [] for L in elL },
#                  hete =   { L: [] for L in hetL },)

    if rhoij_rho2i_pca is None and rho2i_pca is not None:
        rhoij_rho2i_pca = rho2i_pca

    #before = tracemalloc.take_snapshot()
    for f in frames:
        fnat = len(f.numbers)
        frhoi = rhoi[tnat:tnat+fnat]*scale
        fgij = compute_gij(f, spex_ij, hypers_ij)*scale
        for L in range(lmax+1):
            if nu==0:
                lrhonui, lprhonui = np.ones((fnat, 1, 2*L+1)), np.ones((1))
            elif nui==1 or nuj==1:
                lrhonui, lprhonui = compute_rho1i_lambda(frhoi, L, cg)
        
        rho_mp = 
        

In [ ]:
frames,hypers, lmax, nui, nuj, cg = frames, spherical_expansion_hypers, 2, 1, 1, mycg
rhoi_pca=None
rhoij_rho2i_pca=None
rho2i_pca=None
scale=1
tnat = 0
hypers_ij = deepcopy(hypers)
hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)
rhoi = compute_rhoi(frames[:1], spex, hypers)
# els = list(orbs.keys())
# nel = len(els)
for f in frames[:1]:
    fnat = len(f.numbers)
    frhoi = rhoi[tnat:tnat+fnat]*scale
    rhonui, prhonui = frhoi, None
    fgij = compute_gij(f, spex_ij, hypers_ij)*scale
    for L in range(lmax+1):
        #lrho0i, lprho0i = np.ones((fnat, 1, 2*L+1)), np.ones((1))
        lrho1i, lprho1i = compute_rho1i_lambda(frhoi, L, cg)
        
        lrho0ij, prho0ij = compute_rho0ij_lambda(rhonui, fgij, L, cg, prhonui)
        

In [ ]:
np.where(rhonui[:,0,:, lm_slice(2)]-lrho1i)

In [ ]:
lrho0ij

In [ ]:
?compute_rho0ij_lambda

In [ ]:
def compute_rho0ij_lambda(rhoi, gij, L, cg,  prfeats = None): # prfeats is (in analogy with rho2ijlambda) the parity, but is not really necessary)
    """ computes |rho^0_{ij}; lm> """
    rho0ij = gij[..., lm_slice(L)].reshape((gij.shape[0], gij.shape[1], -1, 2*L+1))
    return rho0ij, np.ones(rho0ij.shape[2])

def compute_rho1ij_lambda(rhoj, gij, L, cg, prfeats = None): 
    """ computes |rho^1_{ij}; lm> """

    lmax = int(np.sqrt(gij.shape[-1])) -1
    # can't work out analytically how many terms we have, so we precompute it here
    nl = 0
    for l1 in range(lmax + 1):
        for l2 in range(lmax + 1):  # |rho_i> and |rho_ij; g> are not symmetric so we need all l2
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            nl += 1

    rhoj = rhoj.reshape((rhoj.shape[0], -1, rhoj.shape[-1]))
    # natom, natom, nel*nmax, nmax, lmax+1, lmax+1, M
    shape = (rhoj.shape[0], rhoj.shape[0],
             rhoj.shape[1] , gij.shape[2], nl, 2*L+1)
    rho1jilambda = np.zeros(shape)
    parity = np.ones(nl, dtype = int)*(1-2*(L%2))

    il = 0
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            rho1jilambda[:,:,:,:,il] = cg.combine_einsum(rhoj[...,lm_slice(l1)], gij[...,lm_slice(l2)],
                                                L, combination_string="in,ijN->ijnN")
            parity[il] *= (1-2*(l1%2)) * (1-2*(l2%2))
            il+=1

    return rho1jilambda, parity

In [ ]:
spex = SphericalExpansion(**hypers)
rhoi = compute_rhoi(frames, spex, hypers)
hypers_ij = deepcopy(hypers)
hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)

# Test features and Hamiltonian transformation

In [ ]:
import re
from ase.data import atomic_numbers
import json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [ ]:
# frames = read('qm7/qm7b-chno.xyz', ':100')
# fock = np.load('qm7/qm7-chno-fock.npy', allow_pickle=True)
frames = read('water_random/water_randomized_1000.xyz',':')
for f in frames:
    f.pbc=False
    
focks = np.load('water/water-fock.npy', allow_pickle=True)
orbs = json.loads(json.load(open('water/orbs.json', "r")))

we have to fix the L=1 terms that are stored in a weird order. we do this as post-processing 
and one should undo the change if the matrix is to be read back into pyscf

In [ ]:
for i in range(len(focks)):
    focks[i] = pyscf_fix_l1(focks[i], frames[i], orbs)

## Rotational behavior of features

In [ ]:

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import (WignerDReal, ClebschGordanReal, 
                          spherical_expansion_reshape,
                          lm_slice, real2complex_matrix, xyz_to_spherical, spherical_to_xyz)
from rascal.utils.cg_utils import _r2c as r2c
from rascal.utils.cg_utils import _c2r as c2r
from rascal.utils.cg_utils import _cg as clebsch_gordan
from rascal.utils.cg_utils import _rotation as rotation
from rascal.utils.cg_utils import _wigner_d as wigner_d

### Rotation of features

In [ ]:
frame = frames[90]
frame.cell = [100,100,100]
frame.positions += 50
rotframe = frame.copy()
rotframe.positions = rotframe.positions @ mrot.T
rotframe.cell = rotframe.cell @ mrot.T   # rotate also the cell

feats = spex.transform(frame).get_features(spex)
feats *= 1e6
rfeats = spherical_expansion_reshape(feats, **spherical_expansion_hypers)

g= get_gij_fast(frame, spex,spherical_expansion_hypers)

In [ ]:
rotfeats = spex.transform(rotframe).get_features(spex)
rotfeats *= 1e6
rotfeats = spherical_expansion_reshape(rotfeats, **spherical_expansion_hypers)

rotg = get_gij_fast(rotframe, spex,spherical_expansion_hypers)

Check rotation of $| \overline{\rho_{ij}^0; \lambda \mu} \rangle \equiv \langle n | g; \lambda \mu\rangle $

In [ ]:
np.linalg.norm(WD.rotate(mk_rho0ijlambda_fast(g, 4, mycg)) - mk_rho0ijlambda_fast(rotg, 4, mycg))/np.linalg.norm(mk_rho0ijlambda_fast(g, 4, mycg))

Check rotation of $| \overline{\rho_{i}^1; \lambda \mu} \rangle $

In [ ]:
(np.linalg.norm(WD.rotate(rfeats[...,lm_slice(3)]) - rotfeats[...,lm_slice(3)]))/np.linalg.norm(rfeats[...,lm_slice(3)])

Check $| \overline{\rho_{i}^2; \lambda \mu} \rangle $

In [ ]:
lsoap=mk_rho2ilambda_fast(rfeats, 1, mycg)

In [ ]:
rotlsoap =  mk_rho2ilambda_fast(rotfeats, 1, mycg)
np.linalg.norm(WD.rotate(lsoap) -rotlsoap)/np.linalg.norm(lsoap)

Check rotation of $| \overline{\rho_{ij}^1; 00} \rangle $

In [ ]:
rhoij = mk_rho1ij_fast(rfeats, g, mycg)
rhoij_rot = mk_rho1ij_fast(rotfeats, rotg, mycg)

In [ ]:
np.linalg.norm(WD.rotate(rhoij[...,np.newaxis]) - rhoij_rot[...,np.newaxis])/np.linalg.norm(rhoij)

Check rotation of $| \overline{\rho_{ij}^1; \lambda \mu} \rangle $

In [ ]:
rhoijlm= mk_rho1ijlambda_fast(rfeats, g, 3, mycg)
rhoijlm_rot = mk_rho1ijlambda_fast(rotfeats, rotg, 3, mycg)
np.linalg.norm(WD.rotate(rhoijlm) - rhoijlm_rot)/np.linalg.norm(rhoijlm)

# Simple regression test

In [ ]:
iwater = 99
fock = np.load('water/water-fock.npy', allow_pickle=True)[iwater]
orbs = json.loads(json.load(open("water/orbs.json", "r")))

there has to be a model for each (n1,l1,n2,l2,L) entry in the coupled representation of the Fock matrix

In [ ]:
frame = read('water/water_coords_1000.xyz',':')[iwater]
frame.cell = [100,100,100]
frame.positions += 50
frame.symbols

In [ ]:
fock = pyscf_fix_l1(fock, frame, orbs)
fock_blocks = pyscf_to_blocks(fock, frame, orbs)
fock_blocks_c = to_coupled(fock_blocks, mycg)

In [ ]:
ffeats = do_full_features([frame], orbs, spherical_expansion_hypers, 4, mycg, scale=1e3)

In [ ]:
FR = FockRegression(orbs, alpha=1e-18, solver='svd')

In [ ]:
FR.fit(ffeats, fock_blocks_c)

In [ ]:
fpred = FR.predict(ffeats)

In [ ]:
fpred['diag'][(2,1,2,1)]

In [ ]:
fock_blocks_c['diag'][(2,1,2,1)]

In [ ]:
unfock = blocks_to_pyscf(to_decoupled(fpred, mycg), frame, orbs)

In [ ]:
plt.matshow((unfock-fock).astype(float))
np.linalg.norm(unfock-fock) # bingo!

In [ ]:
np.mean(np.abs(np.linalg.eigvalsh(fock.astype(float))-np.linalg.eigvalsh(unfock)))

In [ ]:
plt.plot(np.linalg.eigvalsh(fock.astype(float)), 'b.')
plt.plot(np.linalg.eigvalsh(unfock.astype(float)), 'r--')

#### Rotation and permutation 

In [ ]:
iwater = 99
frame = read('water/water_coords_1000.xyz',':')[iwater]
frame.cell = [100,100,100]
frame.positions += 50
print(frame.symbols)
fock = np.load('water/water-fock.npy', allow_pickle=True)[iwater]
orbs = json.loads(json.load(open("water/orbs.json", "r")))

In [ ]:
fock = pyscf_fix_l1(fock, frame, orbs)
fock_blocks = pyscf_to_blocks(fock, frame, orbs)
fock_blocks_c = to_coupled(fock_blocks, mycg)
feats = do_full_features([frame], orbs, spherical_expansion_hypers, 4, mycg, scale=1e3)

In [ ]:
FR = FockRegression(orbs, alpha=1e-18, solver='svd')
FR.fit(feats, fock_blocks_c)
fpred = FR.predict(feats)

In [ ]:
fock_original = blocks_to_pyscf(to_decoupled(fpred, mycg), frame, orbs)

In [ ]:
frame_rotperm = frame.copy()
iperm = np.arange(len(frame.numbers), dtype=int)
np.random.shuffle(iperm)
frame_rotperm.numbers = frame_rotperm.numbers[iperm]
frame_rotperm.positions = frame_rotperm.positions[iperm]
print(frame_rotperm.symbols)

abc = np.random.uniform(size=(3))*np.pi
WD = WignerDReal(spherical_expansion_hypers["max_angular"], *abc)
WD.rotate_frame(frame_rotperm)

In [ ]:
feat_rotperm = do_full_features([frame_rotperm], orbs, spherical_expansion_hypers, 4, mycg, scale=1e3)

In [ ]:
pred_rotperm = FR.predict(feat_rotperm)

In [ ]:
fock_rotperm = blocks_to_pyscf(to_decoupled(pred_rotperm, mycg), frame_rotperm, orbs)

In [ ]:
plt.matshow((fock_rotperm-fock_original).astype(float))
np.linalg.norm(fock_rotperm-fock_original) 

In [ ]:
plt.plot(np.linalg.eigvalsh(fock_original.astype(float)), 'b.')
plt.plot(np.linalg.eigvalsh(fock_rotperm.astype(float)), 'r--') 
print(np.mean(np.abs(np.linalg.eigvalsh(fock_original)-np.linalg.eigvalsh(fock_rotperm))))